## Setup: Configure Environment & Imports

In [ ]:
import os
import sys
from pathlib import Path

# Add project to path
project_root = Path('/Users/prasadt1/ai-photography-coach-rag')
sys.path.insert(0, str(project_root))

# Set API key (replace with your own)
os.environ['GOOGLE_API_KEY'] = os.environ.get('GOOGLE_API_KEY', 'YOUR_GEMINI_API_KEY')

print(f"✓ Project root: {project_root}")
print(f"✓ Python path configured")
print(f"✓ API key: {'set' if os.environ.get('GOOGLE_API_KEY') != 'YOUR_GEMINI_API_KEY' else 'NOT SET'}")

## Part 1: Initialize Agents

In [ ]:
import google.generativeai as genai
from agents_capstone.agents.vision_agent import VisionAgent
from agents_capstone.agents.knowledge_agent import KnowledgeAgent
from agents_capstone.agents.orchestrator import Orchestrator

# Configure Gemini
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

# Create agents
vision_agent = VisionAgent()
knowledge_agent = KnowledgeAgent()
orchestrator = Orchestrator(vision_agent, knowledge_agent)

print("✓ VisionAgent initialized")
print("✓ KnowledgeAgent initialized")
print("✓ Orchestrator initialized")

## Part 2: Vision Agent – Analyze a Photo

The VisionAgent uses Gemini Vision to:
1. Extract EXIF metadata (camera, settings)
2. Analyze composition (rule of thirds, focal points, etc.)
3. Identify potential issues (centered subject, underexposure, etc.)

In [ ]:
# For demo, assume an uploaded image exists (from Streamlit app)
# If not, we'll create a placeholder flow

test_image_path = "/Users/prasadt1/ai-photography-coach-rag/tmp_uploaded.jpg"

if os.path.exists(test_image_path):
    print(f"Analyzing image: {test_image_path}")
    vision_result = vision_agent.analyze(test_image_path, skill_level="beginner")
    
    print("\n📊 EXIF Metadata:")
    for key, val in vision_result.exif.items():
        print(f"  {key}: {val}")
    
    print(f"\n📐 Composition Summary:")
    print(f"  {vision_result.composition_summary}")
    
    print(f"\n⚠️  Identified Issues:")
    for issue in vision_result.issues:
        print(f"  - {issue}")
else:
    print(f"❌ Test image not found: {test_image_path}")
    print("\nTo test, upload a photo via the Streamlit app first:")
    print("  python3 -m streamlit run agents_capstone/app_streamlit.py")
    
    # Show a demo structure
    print("\n📋 VisionAnalysis structure (demo):")
    print("""{
  exif: {
    Model: "iPhone 11 Pro Max",
    FocalLength: 4.25,
    FNumber: 1.8,
    ExposureTime: 0.03,
    ISOSpeedRatings: 500
  },
  composition_summary: "The photo has good leading lines...",
  issues: ["subject_centered", "high_iso"]
}""")

## Part 3: Multi-Turn Coaching via Orchestrator

The Orchestrator coordinates agents and manages session state:
1. Loads/creates a session for the user
2. Runs VisionAgent (cached after first call)
3. Runs KnowledgeAgent with session history
4. Persists session for next turn

In [ ]:
# Demo orchestrator flow (if image is available)
if os.path.exists(test_image_path):
    user_id = "demo_user"
    
    # First turn: ask about composition
    query1 = "How can I improve the composition of this photo?"
    print(f"\n🗣️  User: {query1}")
    
    result1 = orchestrator.run(
        user_id=user_id,
        image_path=test_image_path,
        query=query1
    )
    
    coach_text = result1.get("coach", {}).get("text")
    print(f"\n🎯 Coach: {coach_text[:300]}...")
    
    # Second turn: follow-up question (no image needed)
    query2 = "What is ISO and how does it affect image quality?"
    print(f"\n🗣️  User: {query2}")
    
    result2 = orchestrator.run(
        user_id=user_id,
        image_path=None,  # No new image, use session context
        query=query2
    )
    
    coach_text2 = result2.get("coach", {}).get("text")
    print(f"\n🎯 Coach: {coach_text2[:300]}...")
    
    # Show session state
    session = result2.get("session", {})
    print(f"\n📝 Session State:")
    print(f"  User ID: {user_id}")
    print(f"  Skill Level: {session.get('skill_level')}")
    print(f"  History Length: {len(session.get('history', []))} turns")
    if session.get('compact_summary'):
        print(f"  Compact Summary: {session.get('compact_summary')[:100]}...")
else:
    print("(Orchestrator demo requires an uploaded image)")

## Part 4: Session & Memory Management

The ADK adapter transparently manages persistent memory:
- Detects ADK's InMemorySessionService if available
- Falls back to SQLite persistence otherwise
- Stores session state, chat history, and metadata

In [ ]:
from agents_capstone.tools import adk_adapter

print(f"Using ADK: {adk_adapter.USING_ADK}")
print(f"Backend: {'ADK InMemorySessionService' if adk_adapter.USING_ADK else 'SQLite'}")

# Initialize storage
adk_adapter.init()

# Test persistence
test_user = "test_session_user"
test_data = {"skill_level": "advanced", "history": [{"msg": "test"}]}

adk_adapter.set_value(test_user, "session", test_data)
retrieved = adk_adapter.get_value(test_user, "session")

print(f"\n✓ Stored: {test_data}")
print(f"✓ Retrieved: {retrieved}")
print(f"✓ Match: {retrieved == test_data}")

# Show database location
if not adk_adapter.USING_ADK:
    db_path = "/Users/prasadt1/ai-photography-coach-rag/agents_memory.db"
    if os.path.exists(db_path):
        size_mb = os.path.getsize(db_path) / (1024 * 1024)
        print(f"\n📦 Database: {db_path} ({size_mb:.2f} MB)")

## Part 5: Evaluation with LLM-as-Judge

The evaluation harness scores agent responses on:
- **Relevance**: Does it address the user's question?
- **Completeness**: Sufficient detail and context?
- **Accuracy**: Is technical advice correct?
- **Actionability**: Can user act on this immediately?

In [ ]:
from agents_capstone.evaluate import evaluate_sample

# Sample test prompts
test_prompts = [
    "How can I improve the composition of this photo?",
    "What camera settings should I use for a sunset shot?",
    "Explain the rule of thirds and how to apply it.",
]

if os.path.exists(test_image_path):
    print("Running evaluation on test prompts...")
    print("(This may take a minute with LLM-as-Judge scoring)\n")
    
    summary = evaluate_sample(
        image_path=test_image_path,
        prompts=test_prompts,
        out_dir="reports",
        use_llm_judge=True
    )
    
    print(f"\n📊 Evaluation Results:")
    print(f"  Avg Overall Score: {summary['avg_overall_score']}/10")
    print(f"  Avg Latency: {summary['avg_latency_sec']:.2f}s")
    print(f"  Prompts Evaluated: {summary['num_prompts']}")
    
    # Show sample result
    if summary['results']:
        first = summary['results'][0]
        print(f"\n  Sample Result:")
        print(f"    Prompt: {first['prompt']}")
        print(f"    Score: {first['overall_score']}")
        if first['llm_scores']:
            print(f"    Relevance: {first['llm_scores'].get('relevance')}")
else:
    print("(Evaluation demo requires an uploaded image)")
    print("\nTo run evaluation after testing:")
    print("  from agents_capstone.evaluate import evaluate_sample")
    print("  summary = evaluate_sample('path/to/image.jpg', test_prompts)")

## Summary

This notebook demonstrated the **AI Photography Coach** capstone project, showcasing:

✅ **Multi-Agent Architecture** (Day 1)
   - VisionAgent (Gemini Vision + EXIF) → KnowledgeAgent (coaching) via Orchestrator

✅ **Agent Tools** (Day 2)
   - EXIF extraction tool, knowledge base tool
   - Tools structured as ADK-compatible definitions

✅ **Context Engineering** (Day 3)
   - Session management (in-memory + persistent)
   - Multi-turn conversation history
   - Context compaction for long histories

✅ **Agent Quality** (Day 4)
   - Structured logging (JSON format)
   - Observability panel in UI
   - LLM-as-Judge evaluation framework

✅ **Prototype to Production** (Day 5)
   - Streamlit web demo
   - Docker containerization
   - ADK-ready architecture for cloud deployment

## Next Steps

1. **Run the Streamlit app locally:**
   ```bash
   export GOOGLE_API_KEY="YOUR_GEMINI_KEY"
   python3 -m streamlit run agents_capstone/app_streamlit.py
   ```

2. **Run evaluation on a test image:**
   ```bash
   cd agents_capstone
   python3 evaluate.py
   ```

3. **Deploy with Docker:**
   ```bash
   docker build -t photo-coach:latest .
   docker run -e GOOGLE_API_KEY="YOUR_KEY" -p 8501:8501 photo-coach:latest
   ```

## Additional Resources

- 📖 **WRITEUP.md** – Full rubric mapping and submission checklist
- 📋 **ADK_INTEGRATION.md** – ADK setup and integration guide
- 📊 **OBSERVABILITY.md** – Logs, traces, metrics reference
- 🐳 **Dockerfile** – Production-ready container image

---

**Built for:** Google AI Agents Intensive – Capstone Project